In [1]:
from invisible_cities.io import pmap_io
import pandas as pd
import numpy as np
from invisible_cities.reco.xy_algorithms import barycenter
from invisible_cities.database import load_db
from invisible_cities.reco.pmaps_functions import rebin_s2si
from collections import Counter


In [2]:
#find barycenter; first integrated energy over ALL peaks per sipm
def get_barycenter(s2si, DataSiPM):
    peaks=s2si.peak_collection()
    dict_sum=Counter()
    #sum all peaks inside one event
    for peak in peaks:
        dict_sum+=Counter(s2si.peak_and_sipm_total_energy_dict()[peak])

    #map sipm number to coordinates    
    coord=DataSiPM.loc[dict_sum.keys()][['X','Y']]
    ene=[dict_sum[x] for x in dict_sum.keys()]
    X_center,Y_center=barycenter(coord,ene)[0].X,barycenter(coord,ene)[0].Y
    #find sipm that is the closest to the barycenter
    
    return 0.0,0.0
    #return X_center, Y_center



In [3]:
def event_energy(s2si):
    peaks=s2si.peak_collection()
    dict_sum=Counter()
    #sum all peaks inside one event
    for peak in peaks:
        dict_sum+=Counter(s2si.peak_and_sipm_total_energy_dict()[peak])

    #map sipm number to coordinates    
    ene=[dict_sum[x] for x in dict_sum.keys()]
    return ene

In [4]:
def make_xyt_array (s2si, DataSiPM,X_size, Y_size, t_size,t_step, xy_step=10, t_step_round=500):
    X_center,Y_center=get_barycenter(s2si, DataSiPM)
    
    #take as center a sipm to the right up
    X_center,Y_center=DataSiPM[(DataSiPM.X-X_center<xy_step)&(DataSiPM.X-X_center>0)
                               &(DataSiPM.Y-Y_center<xy_step)&(DataSiPM.Y-Y_center>0)].iloc[0][['X','Y']]
    peaks=s2si.peak_collection()
    t_begin=round(s2si.peaks[peaks[0]].tmin_tmax.min//t_step_round)*t_step_round

    x=np.linspace(X_center-X_size*10,X_center+X_size*10,endpoint=False,num=int(X_size*10/5))
    y=np.linspace(Y_center-Y_size*10,Y_center+Y_size*10,endpoint=False,num=int(Y_size*10/5))
    t=np.linspace(t_begin,t_begin+t_step*(t_size-1),int(t_size))
    return x,y,t

In [5]:
from invisible_cities.core.exceptions import *

def E_per_sipm(s2si_rebin,DataSiPM,x,y,time_size,time_step):
    E_xy=np.zeros(time_size)
    try:
        nsipm=np.asscalar(DataSiPM[(DataSiPM.X==x)&(DataSiPM.Y==y)].index)
    except ValueError:
        #print('no sipm')
        return E_xy
    peaks=s2si_rebin.peak_collection()

    t_begin=round(s2si_rebin.peaks[0].t[0]//time_step)*time_step
    for peak in peaks:
        t_start=round(s2si_rebin.peaks[peak].t[0]//time_step)*time_step#s2si_rebin.peaks[peak].t[1]-time_step
        indx_start=int((t_start-t_begin)//time_step)
        indx_end=int(indx_start+len(s2si_rebin.peaks[peak].t))
        try:
            E_xy[indx_start:indx_end]=s2si_rebin.sipm_waveform(peak,nsipm).E
        except ValueError:
            #print('stopped at peak', peak)
            E_xy[indx_start:time_size]=s2si_rebin.sipm_waveform(peak,nsipm).E[:time_size-indx_start]
            break
        except SipmNotFound as ex:
            #print ('{}nsipm not in {}peak'.format(nsipm,peak))
            return E_xy
    return E_xy

def E_tot(s2_rebin,time_size,time_step):
    E=np.zeros(time_size)
    peaks=s2_rebin.peak_collection()

    t_begin=round(s2_rebin.peaks[0].t[0]//time_step)*time_step
    for peak in peaks:
        t_start=round(s2_rebin.peaks[peak].t[0]//time_step)*time_step
        indx_start=int((t_start-t_begin)//time_step)
        indx_end=int(indx_start+len(s2_rebin.peaks[peak].t))
        try:
            E[indx_start:indx_end]=s2_rebin.peak_waveform(peak).E
        except ValueError:
            #print('stopped at peak', peak)
            E[indx_start:time_size]=s2_rebin.peak_waveform(peak).E[:time_size-indx_start]
            break

    return E

In [6]:
def josh_function(file_name,ev_nums,time_step,x_bins,y_bins,t_bins):
    tensor_dict={}
    run,events=pmap_io.read_run_and_event_from_pmaps_file(file_name)
    run_num=np.asscalar(run.run_number.unique())
    #find minimum and maximum event number inside the file and compare with ev_nums:
    ev_min,ev_max=events.evt_number.min(),events.evt_number.max()
    ev_nums=np.array(ev_nums)
    ev_nums_file=ev_nums[(ev_nums>=ev_min) & (ev_nums<=ev_max)]
    #if there are not events in ec_nums return empty dictionary
    if ev_nums_file.size==0:
        return tensor_dict
    
    s1_dict, s2_dict, s2si_dict=pmap_io.load_pmaps(file_name)
    
    DataPMT = load_db.DataPMT (run_num)
    DataSiPM = load_db.DataSiPM(run_num)
    X_min,X_max=DataSiPM.X.min(),DataSiPM.X.max()
    Y_min,Y_max=DataSiPM.Y.min(),DataSiPM.Y.max()
  
    
    
    
    for ev_indx,ev_num in enumerate(ev_nums_file):
        try:
            ev_s=np.asscalar(events[events['evt_number']==ev_num].evt_number.values)
            print('event number {}'.format(ev_s))
        except:
            print('event number {} not found'.format(ev_num))
            continue
        
        #rebin times
        s2_rebin,s2si_rebin=rebin_s2si(s2_dict[ev_s], s2si_dict[ev_s], int(time_step/1000))
    
        x,y,t=make_xyt_array (s2si_rebin, DataSiPM,int(x_bins/2), int(y_bins/2), t_bins,time_step)

        tensor=np.zeros((len(x),len(y),len(t)))
        for ix,vx in enumerate(x):
            for iy,vy in enumerate(y):
                tensor[ix,iy,:]=E_per_sipm(s2si_rebin,DataSiPM,vx,vy,len(t),time_step)
        ene=event_energy(s2si_rebin)
        
        epct = tensor.sum()/np.sum(ene)*100
        print('energy_percent is {}'.format(epct))
        if(epct > 95):

            #normalize xy energies per time slice
           # energy_per_time=(tensor.sum(axis=0)).sum(axis=0)
           # energy_percentage=[1./x if x!=0. else 0. for x in energy_per_time]
           # tensor=tensor*energy_percentage

            #energies form pmts
           # ener= E_tot(s2si_rebin,len(t),time_step)
            #normalize over all times
           # tensor=tensor*ener/ener.sum()
            #alltensors[ev_indx,:,:,:]=tensor
            tensor_dict[ev_num]=tensor
        
    return tensor_dict

In [7]:
def get_real_data(time_step,x_bins,y_bins,t_bins):
    def read_data(loc, rname, ev_nums, f_start=0, f_end=-1):
        path_directory=loc.format(rname)
        files = [join(path_directory, f) for f in listdir(path_directory) if isfile(join(path_directory, f))]
        files=files[f_start:f_end]
        tensor_dict={}
        for fn in files:
            dict_new=josh_function(fn,ev_nums,time_step,x_bins,y_bins,t_bins)
            tensor_dict.update(dict_new)
        return tensor_dict
    return read_data

In [8]:
RUN_NUMBER=4735
from os import listdir
from os.path import isfile, join
data_location='/home/jrenner/analysis/{}/hdf5/pmaps'
read_train=get_real_data(10000,48,48,48)
evtfile = np.load("descape_evts_4735.npz")
evt_list = evtfile["A_evtnum"]



In [9]:
def find_file(loc, rname,ev_number): 
    path_directory=loc.format(rname)
    files = [join(path_directory, f) for f in listdir(path_directory) if isfile(join(path_directory, f))]

    for fn in files:
        run,events=pmap_io.read_run_and_event_from_pmaps_file(fn)
        run_num=np.asscalar(run.run_number.unique())
        #find minimum and maximum event number inside the file and compare with ev_nums:
        
        if events[events.evt_number==ev_number].shape[0]!=0:
            print (fn)
            
    
find_file(data_location,RUN_NUMBER,100936) 

/home/jrenner/analysis/4735/hdf5/pmaps/pmaps_901_4735_icdev_20171006_th_th2000.h5


In [10]:
d=josh_function('/home/jrenner/analysis/4735/hdf5/pmaps/pmaps_901_4735_icdev_20171006_th_th2000.h5',[100936],10000,50,50,50)

event number 100936
energy_percent is 98.91780332610591


In [11]:
all_data=read_train(data_location,RUN_NUMBER,evt_list)

event number 238165
energy_percent is 100.0
event number 310094
energy_percent is 100.0
event number 310102
energy_percent is 100.0
event number 310120
energy_percent is 100.0
event number 115337
energy_percent is 100.0
event number 198076
energy_percent is 100.0
event number 64537
energy_percent is 100.0
event number 223465
energy_percent is 100.0
event number 317591
energy_percent is 100.0
event number 2847
energy_percent is 100.0
event number 113152
energy_percent is 100.0
event number 203711
energy_percent is 100.0
event number 204160
energy_percent is 100.0
event number 76045
energy_percent is 100.0
event number 259562
energy_percent is 100.0
event number 306541
energy_percent is 100.0
event number 314752
energy_percent is 100.0
event number 183386
energy_percent is 100.0
event number 252010
energy_percent is 100.0
event number 252085
energy_percent is 100.0
event number 252102
energy_percent is 100.0
event number 109054
energy_percent is 100.0
event number 221452
energy_percent i

energy_percent is 100.0
event number 52354
energy_percent is 8.733543065657793
event number 90350
energy_percent is 100.0
event number 90383
energy_percent is 100.0
event number 41555
energy_percent is 100.0
event number 41636
energy_percent is 100.0
event number 264641
energy_percent is 100.0
event number 252814
energy_percent is 100.0
event number 137965
energy_percent is 100.0
event number 45044
energy_percent is 100.0
event number 313926
energy_percent is 100.0
event number 34060
energy_percent is 100.0
event number 2945
energy_percent is 100.0
event number 3017
energy_percent is 100.0
event number 18319
energy_percent is 100.0
event number 36924
energy_percent is 100.0
event number 216929
energy_percent is 100.0
event number 121718
energy_percent is 100.0
event number 229656
energy_percent is 100.0
event number 185417
energy_percent is 99.92454535255705
event number 12619
energy_percent is 100.0
event number 145687
energy_percent is 99.2840180669537
event number 25794
energy_perce

energy_percent is 100.0
event number 221269
energy_percent is 100.0
event number 251379
energy_percent is 100.0
event number 215964
energy_percent is 100.0
event number 182930
energy_percent is 100.0
event number 182952
energy_percent is 100.0
event number 215819
energy_percent is 100.0
event number 266645
energy_percent is 100.0
event number 142525
energy_percent is 100.0
event number 145761
energy_percent is 100.0
event number 170699
energy_percent is 100.0
event number 233472
energy_percent is 100.0
event number 8617
energy_percent is 100.0
event number 102039
energy_percent is 100.0
event number 185767
energy_percent is 100.0
event number 310666
energy_percent is 100.0
event number 150299
energy_percent is 100.0
event number 112719
energy_percent is 100.0
event number 112753
energy_percent is 100.0
event number 59259
energy_percent is 100.0
event number 232837
energy_percent is 100.0
event number 315572
energy_percent is 100.0
event number 118482
energy_percent is 100.0
event numbe

event number 71887
energy_percent is 100.0
event number 209881
energy_percent is 100.0
event number 247282
energy_percent is 100.0
event number 311008
energy_percent is 99.96989382279827
event number 129309
energy_percent is 100.0
event number 129328
energy_percent is 100.0
event number 129360
energy_percent is 100.0
event number 65538
energy_percent is 100.0
event number 53165
energy_percent is 100.0
event number 53186
energy_percent is 100.0
event number 160500
energy_percent is 100.0
event number 157066
energy_percent is 100.0
event number 261457
energy_percent is 100.0
event number 74563
energy_percent is 100.0
event number 317373
energy_percent is 100.0
event number 158625
energy_percent is 100.0
event number 198803
energy_percent is 100.0
event number 37063
energy_percent is 100.0
event number 145166
energy_percent is 100.0
event number 59206
energy_percent is 100.0
event number 136084
energy_percent is 100.0
event number 309530
energy_percent is 100.0
event number 121591
energy_

event number 126040
energy_percent is 100.0
event number 66360
energy_percent is 100.0
event number 278794
energy_percent is 4.607132369390572
event number 86032
energy_percent is 100.0
event number 146379
energy_percent is 100.0
event number 279520
energy_percent is 100.0
event number 89781
energy_percent is 100.0
event number 89786
energy_percent is 100.0
event number 141239
energy_percent is 100.0
event number 249276
energy_percent is 100.0
event number 22183
energy_percent is 99.47272526608951
event number 22258
energy_percent is 100.0
event number 22262
energy_percent is 100.0
event number 246110
energy_percent is 100.0
event number 150765
energy_percent is 100.0
event number 186172
energy_percent is 100.0
event number 41497
energy_percent is 100.0
event number 315930
energy_percent is 100.0
event number 261334
energy_percent is 100.0
event number 104463
energy_percent is 100.0
event number 95566
energy_percent is 100.0
event number 178053
energy_percent is 100.0
event number 2149

energy_percent is 100.0
event number 97090
energy_percent is 100.0
event number 287794
energy_percent is 100.0
event number 257556
energy_percent is 100.0
event number 178296
energy_percent is 100.0
event number 268098
energy_percent is 100.0
event number 7891
energy_percent is 99.97053008793722
event number 38732
energy_percent is 99.99327437563691
event number 105791
energy_percent is 100.0
event number 205364
energy_percent is 100.0
event number 158977
energy_percent is 99.02898988341192
event number 158982
energy_percent is 100.0
event number 139094
energy_percent is 100.0
event number 124726
energy_percent is 100.0
event number 269737
energy_percent is 100.0
event number 269784
energy_percent is 100.0
event number 272459
energy_percent is 100.0
event number 125375
energy_percent is 100.0
event number 69265
energy_percent is 100.0
event number 312126
energy_percent is 100.0
event number 8031
energy_percent is 100.0
event number 166379
energy_percent is 100.0
event number 249603
ene

## Export the data to file

In [12]:
import tables as tb

In [13]:
h5maps = tb.open_file("data_4735_nonorm_nocenter.h5", 'w')
filters = tb.Filters(complib='blosc', complevel=9, shuffle=False)
atom_m = tb.Atom.from_dtype(np.dtype('float32'))
maparray = h5maps.create_earray(h5maps.root, 'maps', atom_m, (0, 48, 48, 48), filters=filters)
atom_e = tb.Atom.from_dtype(np.dtype('int'))
evtarray = h5maps.create_earray(h5maps.root, 'evtnum', atom_e, (0, 1), filters=filters)

In [14]:
for k,v in all_data.items():
    evtnum = np.ones(1)*k
    evtarray.append([evtnum])
    maparray.append([v])
h5maps.close()

In [15]:
print(len(all_data))

982


In [ ]:
import pickle


In [ ]:
pickle?

In [ ]:
pickle.dump(all_data,open('pmaps_data.p','wb'))

In [ ]:
%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.patches         import Ellipse

In [ ]:
xdim = 50
ydim = 50

In [ ]:
def NEW_SiPM_map_plot(xarr, normalize=True):
    """
    Plots a SiPM map in the NEW Geometry
    xarr is a NEW sipm map, yarr the pair of coordinates the map corresponds to
    """
    if normalize:
        probs = (xarr - np.min(xarr))
        probs /= np.max(probs)
    else: 
        probs = xarr

    # set up the figure
    fig = plt.figure();
    ax1 = fig.add_subplot(111);
    fig.set_figheight(10.0)
    fig.set_figwidth(10.0)
    ax1.axis([0, 500, 0, 500]);

    for i in range(xdim):
        for j in range(ydim):
            r = Ellipse(xy=(i * 10 + 5, j * 10 + 5), width=5., height=5.);
            r.set_facecolor('0');
            r.set_alpha(probs[i, j]);
            ax1.add_artist(r);
        
    plt.xlabel("x (mm)");
    plt.ylabel("y (mm)");

In [ ]:
NEW_SiPM_map_plot(all_data[1397][:,:,4])

In [ ]:
np.sum(all_data[1397][:,:,0])
svals = []
for key,value in all_data.items():
    svals.append(np.sum(value[:,:,9]))

In [ ]:
hh, bins, patches = plt.hist(svals)

In [ ]:
print(sum(hh))